In [1]:
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

torch.__version__

g:\My Drive\Kaggle\tabular-playground-series-apr-2022\tab22\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'1.10.1+cu113'

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Thu_Nov_18_09:52:33_Pacific_Standard_Time_2021
Cuda compilation tools, release 11.5, V11.5.119
Build cuda_11.5.r11.5/compiler.30672275_0


In [3]:
!nvidia-smi

Mon Apr 11 08:53:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 496.13       Driver Version: 496.13       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   33C    P5    22W / 250W |    829MiB /  8192MiB |     22%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

True
1
0
NVIDIA GeForce RTX 2080 SUPER


In [5]:
data = pd.read_csv('../Data/train_features_computed_tabular.csv')
labels = pd.read_csv('../Data/train_labels.csv')
data = data.merge(labels, on = 'sequence', how = 'left')
X, y = data.drop(['sequence', 'state'], axis = 1).values, data['state'].values

In [6]:
scaler = MinMaxScaler()
model=scaler.fit(X)
X=model.transform(X)

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X,y,random_state = 42)

In [8]:
from pytorch_tabnet.tab_model import TabNetClassifier

In [17]:
clf = TabNetClassifier(
    n_d=64,
    n_a=64,
    n_steps=5,
    gamma=1.5,
    lambda_sparse=1e-4,
    momentum=0.3,
    clip_value=2.0,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    scheduler_params={"gamma": 0.95, "step_size": 20},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    epsilon=1e-15,
)


Device used : cuda


In [18]:
from sklearn.metrics import accuracy_score

In [19]:
clf.fit(
    X_train=X_train,
    y_train=y_train,
    max_epochs=1000, 
    eval_metric=['auc'],
    eval_set=[(X_train, y_train), (X_val, y_val)],
    eval_name=["train", "valid"],
    patience=20,
)


epoch 0  | loss: 1.04016 | train_auc: 0.48176 | valid_auc: 0.50403 |  0:00:02s
epoch 1  | loss: 0.7593  | train_auc: 0.58641 | valid_auc: 0.58802 |  0:00:03s
epoch 2  | loss: 0.69683 | train_auc: 0.60575 | valid_auc: 0.61105 |  0:00:05s
epoch 3  | loss: 0.59447 | train_auc: 0.60723 | valid_auc: 0.59952 |  0:00:07s
epoch 4  | loss: 0.60487 | train_auc: 0.78668 | valid_auc: 0.78355 |  0:00:09s
epoch 5  | loss: 0.59976 | train_auc: 0.77358 | valid_auc: 0.76985 |  0:00:11s
epoch 6  | loss: 0.55619 | train_auc: 0.73651 | valid_auc: 0.73401 |  0:00:13s
epoch 7  | loss: 0.54585 | train_auc: 0.79126 | valid_auc: 0.78501 |  0:00:15s
epoch 8  | loss: 0.53595 | train_auc: 0.60123 | valid_auc: 0.61445 |  0:00:17s
epoch 9  | loss: 0.53294 | train_auc: 0.76899 | valid_auc: 0.76403 |  0:00:19s
epoch 10 | loss: 0.53278 | train_auc: 0.7774  | valid_auc: 0.77224 |  0:00:21s
epoch 11 | loss: 0.534   | train_auc: 0.76964 | valid_auc: 0.76127 |  0:00:23s
epoch 12 | loss: 0.52937 | train_auc: 0.77435 | vali

In [ ]:
#TODO: OPTMA TUNING, ENSEMBLE 